# Initial Tests

In [ ]:
import pandas as pd
import ast
import numpy as np

from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

import os
import openai

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

In [ ]:
# preprocessed file from Thomas
transfers_events = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/combined_data_export.csv')

In [ ]:
# bunch of nan rows
transfers_events = transfers_events[~transfers_events['stay_id'].isna() & ~transfers_events['hadm_id'].isna()]


In [ ]:
# grab subject IDs
transfers_events = transfers_events.merge(edstays[['stay_id', "subject_id"]], on="stay_id", how="left")

In [ ]:
# discharge summaries
discharges = pd.read_csv("/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/discharge.csv.gz")

# ed stays
edstays = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/edstays.csv.gz')

# triage
triage = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/triage.csv.gz')

In [ ]:
triage.stay_id.nunique(), triage.subject_id.nunique(), triage.shape

In [ ]:
# ward transfers
transfers = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/transfers.csv.gz')

# higher-level services (ICU, CARD, etc)
services = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/services.csv.gz')

# get patient info
pts = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/patients.csv.gz')

# admission demographics
admissions = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/admissions.csv.gz')

# procedures
procs = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/procedures_icd.csv.gz')
procs_icd = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/d_icd_procedures.csv.gz')

# diagnoses
diags = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/diagnoses_icd.csv.gz')
diags_icd = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/d_icd_diagnoses.csv.gz')

# meds
med_orders = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/emar.csv.gz')


In [ ]:
med_orders = med_orders[~med_orders['charttime'].isna() & 
                        ~med_orders['medication'].isna() & 
                        ~med_orders['event_txt'].isna()]

In [ ]:
procs = procs.astype({"chartdate":"datetime64[ns]"})
med_orders = med_orders.astype({"charttime":"datetime64[ns]",
                  "scheduletime":"datetime64[ns]",
                  "storetime":"datetime64[ns]",})

discharges = discharges.astype({"charttime":"datetime64[ns]",
                               "storetime":"datetime64[ns]"})

In [ ]:
# are there multiple icd code mappings? 
procs_icd.shape, procs_icd.icd_code.nunique()
# looks like only for the two versions of ICD

In [ ]:
procs_icd[procs_icd['icd_code'] == "067"]

In [ ]:
# drop any potential repeats
procs_icd = procs_icd.groupby(["icd_code", "icd_version"]).first().reset_index()
diags_icd = diags_icd.groupby(["icd_code", "icd_version"]).first().reset_index()

In [ ]:
# grab long_titles for procs/diags
procs = procs.merge(procs_icd, on=["icd_code", "icd_version"], how="left")
diags = diags.merge(diags_icd, on=["icd_code", "icd_version"], how="left")

In [ ]:

# why would someone have multiple ED stays for the same admission?
# looks like they came and left the ED multiple times during the course of their hospital course? 
transfers_events[transfers_events['hadm_id'] == 23880210].iloc[0]['careunit']

In [ ]:
transfers[transfers['hadm_id'] == 29999828].sort_values("intime")[['intime',"outtime",'careunit']]

# Subset Data by Challenge Cohort

## Read in Challenge Data

In [ ]:
import pandas as pd

In [ ]:
challenge_data_fp = "/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/"


In [ ]:
target_train = pd.read_csv(challenge_data_fp + "train/discharge_target.csv.gz", keep_default_na=False)
target_valid = pd.read_csv(challenge_data_fp + "valid/discharge_target.csv.gz", keep_default_na=False)
target_test = pd.read_csv(challenge_data_fp + "test_phase_1/discharge_target.csv.gz", keep_default_na=False)


In [ ]:
target_train

In [ ]:
target_train = pd.read_csv(challenge_data_fp + "train/discharge_target.csv.gz", keep_default_na=False)
target_valid = pd.read_csv(challenge_data_fp + "valid/discharge_target.csv.gz", keep_default_na=False)
target_test = pd.read_csv(challenge_data_fp + "test_phase_1/discharge_target.csv.gz", keep_default_na=False)

discharge_summ_train = pd.read_csv(challenge_data_fp + "train/discharge.csv.gz", keep_default_na=False)
discharge_summ_valid = pd.read_csv(challenge_data_fp + "valid/discharge.csv.gz", keep_default_na=False)
discharge_summ_test = pd.read_csv(challenge_data_fp + "test_phase_1/discharge.csv.gz", keep_default_na=False)

radiology_train = pd.read_csv(challenge_data_fp + "train/radiology.csv.gz", keep_default_na=False)
radiology_valid = pd.read_csv(challenge_data_fp + "valid/radiology.csv.gz", keep_default_na=False)
radiology_test = pd.read_csv(challenge_data_fp + "test_phase_1/radiology.csv.gz", keep_default_na=False)

edstay_train = pd.read_csv(challenge_data_fp + "train/edstays.csv.gz", keep_default_na=False)
edstay_valid = pd.read_csv(challenge_data_fp + "valid/edstays.csv.gz", keep_default_na=False)
edstay_test = pd.read_csv(challenge_data_fp + "test_phase_1/edstays.csv.gz", keep_default_na=False)

edtriage_train = pd.read_csv(challenge_data_fp + "train/triage.csv.gz", keep_default_na=False)
edtriage_valid = pd.read_csv(challenge_data_fp + "valid/triage.csv.gz", keep_default_na=False)
edtriage_test = pd.read_csv(challenge_data_fp + "test_phase_1/triage.csv.gz", keep_default_na=False)

eddiags_train = pd.read_csv(challenge_data_fp + "train/diagnosis.csv.gz", keep_default_na=False)
eddiags_valid = pd.read_csv(challenge_data_fp + "valid/diagnosis.csv.gz", keep_default_na=False)
eddiags_test = pd.read_csv(challenge_data_fp + "test_phase_1/diagnosis.csv.gz", keep_default_na=False)

In [ ]:
target_train['split'] = "train"
target_valid['split'] = "valid"
target_test['split'] = "test"
discharge_summ_train['split'] = "train"
discharge_summ_valid['split'] = "valid"
discharge_summ_test['split'] = "test"
radiology_train['split'] = "train"
radiology_valid['split'] = "valid"
radiology_test['split'] = "test"
edstay_train['split'] = "train"
edstay_valid['split'] = "valid"
edstay_test['split'] = "test"
edtriage_train['split'] = "train"
edtriage_valid['split'] = "valid"
edtriage_test['split'] = "test"
eddiags_train['split'] = "train"
eddiags_valid['split'] = "valid"
eddiags_test['split'] = "test"

In [ ]:
target = pd.concat([target_train, target_valid, target_test])
discharge_summ = pd.concat([discharge_summ_train, discharge_summ_valid, discharge_summ_test])
radiology = pd.concat([radiology_train, radiology_valid, radiology_test])
edstay = pd.concat([edstay_train, edstay_valid, edstay_test])
edtriage = pd.concat([edtriage_train, edtriage_valid, edtriage_test])
eddiags = pd.concat([eddiags_train, eddiags_valid, eddiags_test])

In [ ]:
target = target.reset_index()
discharge_summ = discharge_summ.reset_index()
radiology = radiology.reset_index()
edstay = edstay.reset_index()
edtriage = edtriage.reset_index()
eddiags = eddiags.reset_index()

## Read in MIMIC-IV Data

In [ ]:
# ward transfers
transfers = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/transfers.csv.gz')

# higher-level services (ICU, CARD, etc)
services = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/services.csv.gz')

# get patient info
pts = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/patients.csv.gz')

# admission demographics
admissions = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/admissions.csv.gz')

# procedures
procs = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/procedures_icd.csv.gz')
procs_icd = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/d_icd_procedures.csv.gz')

# diagnoses
diags = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/diagnoses_icd.csv.gz')
diags_icd = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/d_icd_diagnoses.csv.gz')

# meds
med_orders = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/emar.csv.gz')


In [ ]:
# drop any potential repeats
procs_icd = procs_icd.groupby(["icd_code", "icd_version"]).first().reset_index()
diags_icd = diags_icd.groupby(["icd_code", "icd_version"]).first().reset_index()

In [ ]:
# grab long_titles for procs/diags
procs = procs.merge(procs_icd, on=["icd_code", "icd_version"], how="left")
diags = diags.merge(diags_icd, on=["icd_code", "icd_version"], how="left")

In [ ]:
prescriptions = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/prescriptions.csv.gz')
labs = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/labevents.csv.gz')
microbio = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/microbiologyevents.csv.gz')

In [ ]:
labs_icd = pd.read_csv("/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/d_labitems.csv.gz")

In [ ]:
labs = labs.merge(labs_icd, on="itemid", how="left")

## Subset

In [ ]:
cohort_subject_ids = edstay['subject_id'].unique()
cohort_hadm_ids = edstay['hadm_id'].unique()
cohort_stay_ids = edstay['stay_id'].unique()

In [ ]:
len(cohort_subject_ids), len(cohort_hadm_ids), len(cohort_stay_ids)

In [ ]:
transfers = transfers[transfers['subject_id'].isin(cohort_subject_ids)]
services = services[services['subject_id'].isin(cohort_subject_ids)]
pts = pts[pts['subject_id'].isin(cohort_subject_ids)]
admissions = admissions[admissions['hadm_id'].isin(cohort_hadm_ids)]
med_orders = med_orders[med_orders['subject_id'].isin(cohort_subject_ids)]
procs = procs[procs['hadm_id'].isin(cohort_hadm_ids)]
diags = diags[diags['hadm_id'].isin(cohort_hadm_ids)]
prescriptions = prescriptions[prescriptions['hadm_id'].isin(cohort_hadm_ids)]
labs = labs[labs['subject_id'].isin(cohort_subject_ids)]
microbio = microbio[microbio['subject_id'].isin(cohort_subject_ids)]

In [ ]:
transfers.to_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_transfers.pkl')
services.to_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_services.pkl')
pts.to_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_patients.pkl')
admissions.to_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_admissions.pkl')
med_orders.to_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_emar.pkl')
procs.to_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_procedures_icd.pkl')
diags.to_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_diagnoses_icd.pkl')
prescriptions.to_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_prescriptions.pkl')
labs.to_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_labevents.pkl')
microbio.to_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_microbiologyevents.pkl')